# FastAPI Introduction & Fundamentals

Welcome to Module 12! This notebook covers the basics of FastAPI.

## Learning Objectives
- Understand FastAPI basics and async/await
- Create path operations (routes)
- Use path parameters and query parameters
- Understand dependency injection
- Work with request bodies

## What is FastAPI?

FastAPI is a modern Python web framework for building APIs with:
- **Fast**: High performance (comparable to NodeJS and Go)
- **Async**: Built on Starlette, with async/await support
- **Automatic**: Automatic API documentation (Swagger UI)
- **Type hints**: Full Python type hints support
- **Validation**: Automatic request validation with Pydantic

## Part 1: Hello World FastAPI

Let's create a simple FastAPI application.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

# Create FastAPI instance
app = FastAPI(title="ML Model Registry", version="1.0.0")

# Define a route
@app.get("/")
async def root():
    """Root endpoint that returns a greeting"""
    return {"message": "Welcome to ML Model Registry!"}

# In production, you'd run with: uvicorn app:app --reload
# But here we just show the function
print("✅ FastAPI app created!")
print("Route: GET / → Returns greeting")

## Part 2: Path Parameters

Path parameters are dynamic parts of the URL.

In [ ]:
# Example: GET /models/123 returns model with ID 123
# The {model_id} is a path parameter

@app.get("/models/{model_id}")
async def get_model(model_id: int):
    """
    Get a model by ID.
    
    Args:
        model_id: The ID of the model (must be an integer)
    
    Returns:
        Model details dictionary
    """
    return {
        "model_id": model_id,
        "name": f"Model {model_id}",
        "status": "active"
    }

# Test it
import asyncio
result = await get_model(123)
print(f"✅ Path parameter example:")
print(f"GET /models/123 → {result}")

## Part 3: Query Parameters

Query parameters come after the `?` in the URL.

In [ ]:
# Example: GET /models?skip=0&limit=10

@app.get("/models/")
async def list_models(skip: int = 0, limit: int = 10):
    """
    List models with pagination.
    
    Args:
        skip: Number of models to skip (default: 0)
        limit: Maximum models to return (default: 10)
    
    Returns:
        List of models with pagination info
    """
    models = [
        {"id": i, "name": f"Model {i}", "accuracy": 0.8 + i * 0.01}
        for i in range(1, 21)
    ]
    
    # Apply pagination
    paginated = models[skip : skip + limit]
    
    return {
        "total": len(models),
        "skip": skip,
        "limit": limit,
        "items": paginated
    }

# Test it
result = await list_models(skip=0, limit=3)
print(f"✅ Query parameters example:")
print(f"GET /models?skip=0&limit=3 → {len(result['items'])} models returned")

## Part 4: Request Body with Pydantic

Pydantic automatically validates request bodies.

In [ ]:
from pydantic import BaseModel
from typing import Optional

# Define request schema
class ModelCreate(BaseModel):
    name: str
    framework: str  # sklearn, pytorch, tensorflow
    accuracy: Optional[float] = None
    
    class Config:
        json_schema_extra = {
            "example": {
                "name": "My ML Model",
                "framework": "sklearn",
                "accuracy": 0.95
            }
        }

@app.post("/models/")
async def create_model(model: ModelCreate):
    """
    Create a new ML model.
    
    Args:
        model: Model data (validated by Pydantic)
    
    Returns:
        Created model with ID
    """
    return {
        "id": 123,
        "name": model.name,
        "framework": model.framework,
        "accuracy": model.accuracy,
        "status": "created"
    }

# Test it
test_model = ModelCreate(
    name="Test Model",
    framework="pytorch",
    accuracy=0.92
)
result = await create_model(test_model)
print(f"✅ Request body example:")
print(f"POST /models/ → {result}")

## Part 5: HTTP Methods

Different HTTP methods for different operations.

In [ ]:
# GET - Retrieve data
@app.get("/models/{model_id}")
async def read_model(model_id: int):
    return {"model_id": model_id, "status": "read"}

# POST - Create data
@app.post("/models/")
async def create_model_v2(model: ModelCreate):
    return {"id": 123, **model.dict(), "status": "created"}

# PUT - Update entire resource
@app.put("/models/{model_id}")
async def update_model(model_id: int, model: ModelCreate):
    return {"model_id": model_id, **model.dict(), "status": "updated"}

# DELETE - Remove resource
@app.delete("/models/{model_id}")
async def delete_model(model_id: int):
    return {"model_id": model_id, "status": "deleted"}

print("✅ HTTP Methods defined:")
print("  GET /models/{id} - Read")
print("  POST /models/ - Create")
print("  PUT /models/{id} - Update")
print("  DELETE /models/{id} - Delete")

## Part 6: Dependency Injection

FastAPI's dependency system makes code reusable and testable.

In [ ]:
from fastapi import Depends

# Define a dependency
async def get_current_user() -> dict:
    """
    Simulated dependency that returns current user.
    In production, this would validate JWT tokens.
    """
    return {
        "user_id": 1,
        "username": "john_doe",
        "email": "john@example.com"
    }

# Use the dependency in a route
@app.get("/profile")
async def get_profile(current_user: dict = Depends(get_current_user)):
    """
    Get current user's profile.
    Automatically injects the user from get_current_user dependency.
    """
    return {
        "profile": current_user,
        "models_count": 5,
        "experiments_count": 3
    }

# Test it
result = await get_profile()
print(f"✅ Dependency injection example:")
print(f"GET /profile → {result['profile']['username']}'s profile loaded")

## Part 7: Status Codes and Responses

Control HTTP status codes and response details.

In [ ]:
from fastapi import HTTPException, status

@app.post("/models/", status_code=status.HTTP_201_CREATED)
async def create_model_with_status(model: ModelCreate):
    """
    Create model and return 201 Created status.
    """
    return {
        "id": 456,
        **model.dict(),
        "status": "created"
    }

@app.get("/models/{model_id}")
async def get_model_or_error(model_id: int):
    """
    Get model or raise 404 Not Found.
    """
    if model_id < 1:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Model {model_id} not found"
        )
    
    return {"model_id": model_id, "name": f"Model {model_id}"}

print("✅ Status codes:")
print("  201 Created - Resource created")
print("  404 Not Found - Resource doesn't exist")
print("  200 OK - Success")
print("  400 Bad Request - Invalid input")

## Part 8: Async/Await in FastAPI

FastAPI is built for async operations - perfect for I/O bound tasks like database queries.

In [ ]:
import asyncio
import time

# Simulate async database operation
async def fetch_from_database(model_id: int) -> dict:
    """
    Simulate fetching from database.
    In real apps, this is where async database drivers shine.
    """
    print(f"  → Starting to fetch model {model_id}...")
    await asyncio.sleep(0.5)  # Simulate I/O delay
    print(f"  → Finished fetching model {model_id}")
    return {
        "id": model_id,
        "name": f"Model {model_id}",
        "accuracy": 0.95
    }

# Async route - can handle multiple concurrent requests
@app.get("/models/async/{model_id}")
async def get_model_async(model_id: int):
    """
    Async endpoint that fetches from database.
    FastAPI can handle 1000s of concurrent requests efficiently.
    """
    model = await fetch_from_database(model_id)
    return model

# Test async operations
print("✅ Async/Await example:")
start = time.time()

# Sequential (slow)
print("Sequential:")
for i in [1, 2]:
    await fetch_from_database(i)
seq_time = time.time() - start
print(f"  Time: {seq_time:.1f}s\n")

# Concurrent (fast) - using asyncio.gather
print("Concurrent:")
start = time.time()
await asyncio.gather(
    fetch_from_database(1),
    fetch_from_database(2)
)
conc_time = time.time() - start
print(f"  Time: {conc_time:.1f}s")
print(f"\n💡 Async is {seq_time/conc_time:.1f}x faster for concurrent I/O!")

## Key Takeaways

1. **FastAPI is fast** - Built on modern async/await Python
2. **Automatic validation** - Pydantic validates all inputs
3. **Auto documentation** - OpenAPI docs generated automatically
4. **Type hints** - Full type support for better IDE experience
5. **Dependency injection** - Reusable, testable components
6. **Async ready** - Built for high concurrency

## Next Steps

1. Run the full application: `docker-compose up -d`
2. Visit http://localhost:8000/docs for interactive API explorer
3. Study the next notebook: Pydantic Validation
4. Explore the actual implementation in `app/api/v1/`

## Resources

- [FastAPI Official Docs](https://fastapi.tiangolo.com/)
- [Async/Await Guide](https://docs.python.org/3/library/asyncio.html)
- [HTTP Status Codes](https://httpwg.org/specs/rfc7231.html#status.codes)